In [1]:
from configs import models, env
from backend.llm.api import load_llm_langchain

config_loaded = {"model_config": models, "env": env}
llm = load_llm_langchain(source='groq', model_name='LLaMA-3', config=config_loaded)

[LLM Loader] Successfully initialized model 'llama-3.1-8b-instant' from 'groq'.


In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from backend.vector_db.clients import get_vector_retriever
from backend.agents.nodes import (
    symptom_node,
    ehr_node,
    literature_node,
    drug_node,
    treatment_node,
    inject_retrieved_context
)

# ✨ Conditional Flow Setup
def condition_ehr(state):
    return state.get("ehr_text") is not None

def condition_question(state):
    return state.get("question") is not None

# --- 📊 Agent State Schema ---
class AgentState(TypedDict):
    symptoms: Optional[str]
    question: Optional[str]
    ehr_text: Optional[str]
    medications: Optional[list[str]]
    patient_profile: Optional[str]
    diagnosis: Optional[str]
    summary: Optional[str]
    literature_answer: Optional[str]
    interaction_report: Optional[str]
    treatment_plan: Optional[str]
    context: Optional[str]

# --- 🧠 Load LLM and Retriever ---
retriever = get_vector_retriever(
    store_type="chroma",  # or "faiss"
    persist_dir="/Users/sunnyraj/code_files/git_repos/MedAgenticSage/data/storage/chroma",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)
# --- 🧩 LangGraph Flow ---
builder = StateGraph(AgentState)

# --- 🔁 Nodes with Retriever ---
builder.add_node("symptom_checker", lambda state: symptom_node(state, llm, retriever))
builder.add_node("ehr_summarizer", lambda state: ehr_node(state, llm, retriever))
builder.add_node("literature_qa", lambda state: literature_node(state, llm, retriever))
builder.add_node("drug_checker", lambda state: drug_node(state, llm, retriever))
builder.add_node("treatment_planner", lambda state: treatment_node(state, llm, retriever))
builder.add_node("inject_context", lambda state: inject_retrieved_context(state, retriever))

# --- 🧠 Conditional Flow ---
builder.set_entry_point("inject_context")
builder.add_conditional_edges("inject_context", condition_question, {
    True: "literature_qa",
    False: "symptom_checker"
})
builder.add_conditional_edges("symptom_checker", condition_ehr, {
    True: "ehr_summarizer",
    False: "drug_checker"
})
builder.add_edge("ehr_summarizer", "drug_checker")
builder.add_edge("literature_qa", "drug_checker")
builder.add_edge("drug_checker", "treatment_planner")
builder.add_edge("treatment_planner", END)

# --- 🚀 Compile and Run ---
app_graph = builder.compile()

/Users/sunnyraj/code_files/git_repos/MedAgenticSage/backend/vector_db/clients.py:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/Users/sunnyraj/code_files/git_repos/MedAgenticSage/backend/vector_db/clients.py:36: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.store = Chroma(


In [3]:
initial_state = {
    "symptoms": "high fever, chills, cough",
    "ehr_text": "Patient reports persistent cough with fatigue. Chest X-ray shows mild inflammation..",
    "question": "What recent treatments are effective for respiratory infections?",
    "medications": ["paracetamol", "azithromycin"],
    "patient_profile": {
        "diagnosis": "no chronic illnesses",
        "age": 45,
        "sex": "Male",
        "comorbidities": []
    }
}

final_state = app_graph.invoke(initial_state)

# %%
for key, value in final_state.items():
    print(f"\n📌 {key.upper()}:")
    if isinstance(value, str):
        print(value)
    elif hasattr(value, "content"):
        print(value.content)
    elif isinstance(value, list):
        for i, item in enumerate(value, 1):
            print(f"  {i}. {item}")
    else:
        print(repr(value))

🔑 Keys in state at literature_node: ['symptoms', 'question', 'ehr_text', 'medications', 'patient_profile', 'context']
🔑 Keys in state at drug_node: ['symptoms', 'question', 'ehr_text', 'medications', 'patient_profile', 'literature_answer', 'context']
🔑 Keys in state at treatment_node: ['symptoms', 'question', 'ehr_text', 'medications', 'patient_profile', 'literature_answer', 'interaction_report', 'context']

📌 SYMPTOMS:
high fever, chills, cough

📌 QUESTION:
What recent treatments are effective for respiratory infections?

📌 EHR_TEXT:
Patient reports persistent cough with fatigue. Chest X-ray shows mild inflammation..

📌 MEDICATIONS:
  1. paracetamol
  2. azithromycin

📌 PATIENT_PROFILE:
{'diagnosis': 'no chronic illnesses', 'age': 45, 'sex': 'Male', 'comorbidities': []}

📌 LITERATURE_ANSWER:
Based on the latest PubMed literature, here are some recent treatments that have been found to be effective for respiratory infections:

1. **Antiviral medications for influenza**:
   - Oseltamivi